# Обработка текстов

Вам дан файл `ru.conversations.txt`

В нем содержатся диалоги (разделены пустой строкой).

Вам необходимо:

### Разбить текст на диалоги

In [1]:
with open('ru.conversations.txt', 'r', encoding='utf-8') as f:
    text = []
    for line in f:
        text.append(line)

In [2]:
text

['- Что,  Мансур,  не жарко теперь тебе?\n',
 '- Спрашиваю, не жарко ему теперь?\n',
 '\n',
 '- Какой полк?\n',
 '- Тысяча тридцать четвертый.\n',
 '- Вези дальше. Тут тысяча двадцать шестой.\n',
 '\n',
 '- Какая санрота?\n',
 '- Тысяча тридцать шестая.\n',
 '- Значит, наша! Принимай тяжелораненого!\n',
 '\n',
 '- Сколько фрицев в котле?\n',
 '- Тысяч сорок.\n',
 '\n',
 '- Чем же он отравился?\n',
 '- А вон, видишь, что-то из тех бутылей выпил.\n',
 '\n',
 '- Ну что, Мансур, как дела?\n',
 '- Фашисты прорвали оборону.\n',
 '- А мы с тобой спим?!\n',
 '\n',
 '- Куда их?\n',
 '- В Гумрак.\n',
 '\n',
 '- Комсорг, а не против, если еще по сто?\n',
 '- Можно.\n',
 '\n',
 '- Где брали-то?\n',
 '- Во-он.\n',
 '\n',
 '- Мансур, ведь ты его сбил!\n',
 '- А может, кто-нибудь еще стрелял?\n',
 '\n',
 '- А где комбат?\n',
 '- Унесли. Ранило его.\n',
 '\n',
 '- Что, сынок, на лошадей больше надежда?\n',
 '- Да,  да.\n',
 '\n',
 '- Кто вы, отец?\n',
 '- Священник.\n',
 '\n',
 '- А напысав хто-нэбудь

In [3]:
dialogs = []
dialog = ''
for item in text:
    if item != '\n':
        dialog += item
    else:
        dialogs.append(dialog)
        dialog = ''
print(dialogs[:5])

['- Что,  Мансур,  не жарко теперь тебе?\n- Спрашиваю, не жарко ему теперь?\n', '- Какой полк?\n- Тысяча тридцать четвертый.\n- Вези дальше. Тут тысяча двадцать шестой.\n', '- Какая санрота?\n- Тысяча тридцать шестая.\n- Значит, наша! Принимай тяжелораненого!\n', '- Сколько фрицев в котле?\n- Тысяч сорок.\n', '- Чем же он отравился?\n- А вон, видишь, что-то из тех бутылей выпил.\n']


### Разбить диалоги на токены

In [6]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pro10\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
for i,item in enumerate(dialogs):
    dialogs[i] = word_tokenize(item)

### Предобработать токены
Удалить стоп-слова и спецсимволы, лемматизировать или стеммировать слова и т.д.

In [9]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pro10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
tokenizer = RegexpTokenizer(r'\w+')
for i,item in enumerate(dialogs):
    dialogs[i] = tokenizer.tokenize(str(dialogs[i]))

In [12]:
clean_dialogs = dialogs
for i, dialog in enumerate(dialogs):
    clean_dialog = dialog
    for word in dialog:
        if word.lower() in stopwords.words('russian'):
            clean_dialog.remove(word)
    clean_dialogs[i] = clean_dialog

In [13]:
clean_dialogs[:5]

[['Мансур', 'жарко', 'тебе', 'Спрашиваю', 'жарко', 'теперь'],
 ['полк',
  'Тысяча',
  'тридцать',
  'четвертый',
  'Вези',
  'дальше',
  'тысяча',
  'двадцать',
  'шестой'],
 ['санрота',
  'Тысяча',
  'тридцать',
  'шестая',
  'Значит',
  'наша',
  'Принимай',
  'тяжелораненого'],
 ['Сколько', 'фрицев', 'котле', 'Тысяч', 'сорок'],
 ['же', 'отравился', 'вон', 'видишь', 'то', 'тех', 'бутылей', 'выпил']]

In [14]:
from pymystem3 import Mystem
mystem = Mystem()

In [19]:
%%timeit
mystem.lemmatize('Котле')

1.24 s ± 12.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Лемматизация русских слов займет много времени

In [ ]:
#lemma_dialogs = clean_dialogs
#for i, dialog in enumerate(clean_dialogs):
#    lemma_dialog = dialog
#    for j, word in enumerate(dialog):
#        lemma_dialog[j] = mystem.lemmatize(word)[0]
#    lemma_dialogs[i] = lemma_dialog

In [24]:
from nltk.stem import SnowballStemmer
rus_stemmer = SnowballStemmer('russian')

In [26]:
%%timeit
rus_stemmer.stem('котле')

76.4 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [29]:
stem_dialogs = clean_dialogs
for i, dialog in enumerate(clean_dialogs):
    stem_dialog = dialog
    for j, word in enumerate(dialog):
        stem_dialog[j] = rus_stemmer.stem(word)
    stem_dialogs[i] = stem_dialog

### Перевести каждый из диалогов в векторное представление

С помощью TF-IDF представить каждый из диалогов в векторном формате

### Найти самые близкие по содержанию диалоги, исходя из векторного представления